In [2]:
import pandas as pd
import gzip
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [4]:
# possible values: "Electronics", "Cell_Phones_and_Accessories", "Clothing_Shoes_and_Jewelry"
product_type = "Electronics"
meta_fname = "meta_" + product_type + ".json.gz"
df = getDF(meta_fname)
df.shape

(498196, 9)

In [5]:
df.columns

Index(['asin', 'imUrl', 'description', 'categories', 'title', 'price',
       'salesRank', 'related', 'brand'],
      dtype='object')

In [6]:
key_cols = ['asin', 'title', 'price', 'brand','categories', 'description']

In [7]:
df[['asin','categories']].sample(10)  
# categories are nested lists, each list is one category and its sub-categories under all electronics

,asin,categories
408673,B00B1NAAKU,"[[Electronics, Computers & Accessories, Cables & Accessories, Computer Cable Adapters, USB-to-USB Adapters]]"
148956,B002KDQPWM,"[[Electronics, Camera & Photo, Accessories, Digital Camera Accessories, Shutter Release Cords]]"
16464,B00008YMEV,"[[Electronics, Computers & Accessories, Cables & Accessories, Surge Protectors]]"
481124,B00H9ORVRK,"[[Electronics, Computers & Accessories, Laptops]]"
456588,B00EN27ULS,"[[Electronics, Computers & Accessories, Cables & Accessories, Input Devices, Graphics Tablets]]"
309991,B006SF6D1Q,"[[Electronics, Computers & Accessories, Data Storage, USB Flash Drives]]"
92128,B0011DK0GY,"[[Electronics, Car & Vehicle Electronics, Vehicle Electronics Accessories, Radar Detectors]]"
438795,B00D23K2Q4,"[[Electronics, Computers & Accessories, Laptops]]"
201216,B003VW22UK,"[[Electronics, Accessories & Supplies, Audio & Video Accessories, TV Accessories & Parts, TV Ceiling & Wall Mounts]]"
440956,B00D8XNU6C,"[[Electronics, Accessories & Supplies, Audio & Video Accessories, Remote Controls, TV Remote Controls]]"


In [8]:
from collections import defaultdict
cat = defaultdict(int)

for row in df['categories']:
    for e in row:
        hc = ''
        for i,c in enumerate(e):
            if i > 0:
                hc = hc + '->' + c
            else:
                hc = c
            cat[hc] += 1

print(len(cat.keys()))

1567


In [9]:
cat_df = pd.DataFrame.from_dict(cat, orient = 'index')
cat_df.reset_index(level = 0, inplace = True)
cat_df.columns = ['category','count']
cat_df['levels'] = cat_df['category'].apply(lambda x: len(x.split('->')))
cat_df.sample(5)

,category,count,levels
578,Electronics->Car & Vehicle Electronics->Car Electronics->Car Audio->Amplifiers,1842,5
911,Electronics->Computers & Accessories->Warranties & Services->Service Plans,3,4
1313,Sports & Outdoors->Accessories->Sport Watches,5,3
597,Electronics->Computers & Accessories->External Components->External TV Tuners,77,4
35,Electronics->GPS & Navigation->Sports & Handheld GPS,1214,3


In [10]:
import numpy as np
np.max(cat_df['levels'])

7

In [11]:
cat_df_sorted = cat_df.sort_values(by = ['levels','count'],ascending=[True, False])
cat_df_sorted.to_csv(product_type +"_category_counts.csv")

In [25]:
# extract the meta data for example category
select_cat = 'Electronics->Accessories & Supplies->Audio & Video Accessories->TV Accessories & Parts->TV Ceiling & Wall Mounts'.split('->')
select_meta = df[ df['categories'].apply(lambda x: select_cat in x) ]
select_meta.shape

(3540, 9)